# Trajectory 사람 이동 예측
trajectory 데이터에서 총 180명 정도의 데이터가 존재 <br>
개인 별로 어떻게 움직일지를 lstm, gru (파라미터가 좀 작을 때 사용하는 모델),transfomer 모형을 활용해서 학습해볼 예정 <br>
## 표본 선정 기준<br>
1. 수집 일 수가 많을 것 <br>
2. 일정하게 데이터가 존재할 것 <br>
결과적으로 총 6명을 학습 할 예정이다. <br>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd DeepLearning_14Team/

/content/drive/MyDrive/DeepLearning_14Team


In [5]:
import os
import pandas as pd
list_ = os.listdir('sorted_data/128/Trajectory/')
sorted_41 = []
for i in range(len(list_)):
    file_ = open('sorted_data/128/Trajectory/' + list_[i], 'r') 
    file = file_.readlines()
    file = file[6:]
    for j in range(len(file)):
        tmp = file[j].split(',')
        sorted_41.append([float(tmp[0]), float(tmp[1]), tmp[-2] + ' ' +tmp[-1][:-1]])

df = pd.DataFrame(sorted_41)
df[2] = pd.to_datetime(df[2])
df.sort_values(by = [2],inplace=True)
df = df.reset_index()
df = df[[0,1,2]]

In [ ]:
len(df)

1208500

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import pandas as pd
df = pd.read_csv('sorted_128_last_1000.csv')

# 데이터 분리하기
1. 데이터가 너무 많다는 점
2. 어느 곳에 방문할지에 대한 정보가 좀 더 중요하다는 점
3. 사람이 걸을 수 있는 정도가 한정되어 있다는 점 <br>
세가지를 고려할 때 모두를 학습시키기 보다는 일부분을 쪼개서 학습시키는 것이 합리적

In [ ]:
result / len(a)

Timedelta('0 days 00:01:27.645094826')

# 30분 단위로 쪼개서 이 사람의 미래 예측하는 형태
    평균이 1분 27초 정도되니 30정도 곱해서 30분 단위로 예측해볼 것임

train, test, val set으로 분리하였다. <br>
시간이 중요한게 아니라 순서가 중요하다고 할 수 있는건가? <br>
균일하게 되어 있는가?

In [6]:
df_diff_30_train = []
for i in range(0,len(df),30):
    df_diff_30_train.append(df.iloc[i])
df_diff_30_val = []
for i in range(10, len(df), 30):
    df_diff_30_val.append(df.iloc[i])
df_diff_30_test = []
for i in range(20, len(df), 30):
    df_diff_30_test.append(df.iloc[i])

In [ ]:
len(df_diff_30)

37028

In [7]:
df_diff_30_train_pd = pd.DataFrame(df_diff_30_train)
df_diff_30_test_pd = pd.DataFrame(df_diff_30_test)
df_diff_30_val_pd = pd.DataFrame(df_diff_30_val)

In [ ]:
x, y = df_diff_30_train_pd['0'], df_diff_30_train_pd['1']

In [ ]:
y

0      116.327817
30     116.321833
60     116.323883
90     116.320683
120    116.325833
150    116.310317
180    116.311717
210    116.309617
240    116.309717
270    116.313667
300    116.322567
330    116.328100
360    116.329650
390    116.327583
420    116.321033
450    116.322983
480    116.297283
510    116.282217
540    116.262100
570    116.243333
600    116.232183
630    116.222750
660    116.250917
690    116.262900
720    116.268900
750    116.262883
780    116.294067
810    116.304850
840    116.301350
870    116.307233
900    116.310017
930    116.301583
960    116.291483
990    116.297633
Name: 1, dtype: float64

# 30분 단위로 쪼갠다!

In [ ]:
df_128_30 = pd.DataFrame(df_diff_30)

In [9]:
train_x, train_y = df_diff_30_train_pd[0], df_diff_30_train_pd[1]
test_x, test_y = df_diff_30_test_pd[0], df_diff_30_test_pd[1]
val_x, val_y = df_diff_30_val_pd[0], df_diff_30_val_pd[1]

# 모델 학습 데이터 준비
    x, y 차원을 맞추는 것이 어려운 점이었음
    lstm 모델 2개를 활용해서 한 차원 당 1개의 모델을 사용하도록 하였음.
    

In [ ]:
x

0      40.075850
1      40.075833
2      40.076000
3      40.076300
4      40.076600
         ...    
995    40.255033
996    40.255067
997    40.255217
998    40.255483
999    40.255733
Name: 0, Length: 1000, dtype: float64

In [11]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences) - n_steps):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-2], sequences[end_ix, -2:]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# train x, y
# define input sequence
in_seq1 = train_x.values
in_seq2 = train_y.values
#out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
#out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
train_dataset = hstack((in_seq1, in_seq2, in_seq1, in_seq2))

# test x, y
# define input sequence
in_seq1 = train_x.values
in_seq2 = train_y.values
#out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
#out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
test_dataset = hstack((in_seq1, in_seq2, in_seq1, in_seq2))

# val x, y
# define input sequence
in_seq1 = train_x.values
in_seq2 = train_y.values
#out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
#out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
val_dataset = hstack((in_seq1, in_seq2, in_seq1, in_seq2))

In [12]:
df_train_x, df_train_y  = split_sequences(train_dataset, 5)
df_test_x, df_test_y = split_sequences(test_dataset, 5)
df_val_x, df_val_y = split_sequences(val_dataset, 5)

# lstm 모델 학습

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
import torch

In [15]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 100 # number of hidden states
        self.n_layers = 300 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                batch_first = True,dropout=0.1).to(device)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1).to(device)
        self.relu = torch.nn.ReLU()
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size, self.n_hidden, device= device)
        cell_state = torch.zeros(self.n_layers,batch_size, self.n_hidden, device = device)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x, self.hidden)
        lstm_out = self.relu(lstm_out)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [16]:
n_features = 1 # this is number of parallel inputs
n_timesteps = 5 # this is number of timesteps

# convert dataset into input/output
df_train = split_sequences(train_dataset, 5)
df_test = split_sequences(test_dataset, 5)
df_val = split_sequences(val_dataset, 5)

# create NN
mv_net_x = MV_LSTM(n_features, n_timesteps)
mv_net_y = MV_LSTM(n_features, n_timesteps)
criterion = torch.nn.MSELoss().to(device=device)
 # reduction='sum' created huge loss value
optimizer_x = torch.optim.Adam(mv_net_x.parameters(), lr=1e-1)
optimizer_y = torch.optim.Adam(mv_net_y.parameters(), lr=1e-1)

# scheduler
scheduler_x = torch.optim.lr_scheduler.StepLR(optimizer_x, 1.0, gamma=0.95)
scheduler_y = torch.optim.lr_scheduler.StepLR(optimizer_y, 1.0, gamma=0.95)

# MV_LSTM(n_features, n_timesteps) # n_hidden, n_layers
mv_net_test = MV_LSTM(n_features, n_timesteps)
layer = mv_net_test.n_layers
hidden = mv_net_test.n_hidden

train_episodes = 30
batch_size = 50

In [ ]:
type(X) 
# 여기서 타입 에러가 발생했음 # float 타입으로 결과를 만들어야 했는데
# 위에서 string 타입으로 데이터 프레임을 만들어내서 문제가 발생

numpy.ndarray

# 여기서도 어려웠던 점
    인풋 사이즈를 맞춰줘야 하는데 그 자체가 코드를 모두 이해하여야 했기에 엄청난 어려움이 있었다.
    ★시계열 데이터이기 때문에 마지막 배치의 경우에 문제가 존재했음 -> 이 부분을 처리하는 것이 중요!
    텐서 자체에서 문제가 존재했고, 데이터를 텐서로 변환하는 과정에서 소수점의 사라짐이 존재하게 되었음 이러한 문제점을 해결하는 방법은?


In [ ]:
처음에 트레인, 테스트, 벨리데이션 셋을 나누지 않고 모델을 만들었다.
그러나 이후에 이 모델이 잘 만들어 졌는지를 확인하기 위해서 찾아보니 다음과 같은 방식으로 처리한다는 것을 알게 되었다.
이후에 코드를 수정하여 메트릭을 평가할 수 있었다.
그리고 10분, 20분, 30분 부터 시작하여 30분씩 미래 데이터를 예측해보고자 하였음

In [17]:
torch.set_printoptions(precision=10)

In [25]:
def train():
    mv_net_x.train()
    mv_net_y.train()
    start_time = time.time()
    total_loss = 0.

    for b in range(0,len(df_train_x),batch_size):
        inpt = df_train_x[b:b+batch_size,:,:]
        target = df_train_y[b:b+batch_size]

        # x, y 트레인셋
        x_batch = torch.tensor(inpt[:,:,0],dtype=torch.float32, device = device) # 텐서로 바꾸면서
        y_batch = torch.tensor(inpt[:,:,1],dtype=torch.float32, device = device)
        
        if x_batch.shape == (batch_size,n_timesteps):
            x_batch = x_batch.reshape(batch_size,n_timesteps,1) # 3 채널로 맞춰줘야 함
            y_batch = y_batch.reshape(batch_size,n_timesteps,1)

            mv_net_x.init_hidden(x_batch.size(0))
            mv_net_y.init_hidden(y_batch.size(0))

            # x,y의 예측 값
            output_x = mv_net_x(x_batch)
            output_y = mv_net_y(y_batch)

            # x,y의 정답
            target_x_batch = torch.tensor(target[:, 0],dtype=torch.float32, device = device)
            target_y_batch = torch.tensor(target[:, 1],dtype=torch.float32, device = device)


        #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
        #    lstm_out.contiguous().view(x_batch.size(0),-1)

            # loss function이 두개의 합이 최소가 되는 쪽으로 loss를 구함
            loss = criterion(output_x.view(-1), target_x_batch) + criterion(output_y.view(-1), target_y_batch)
            loss.backward()

            optimizer_x.step()        
            optimizer_x.zero_grad() # x가 0으로 흐르도록 함

            optimizer_y.step()
            optimizer_y.zero_grad() # y가 0으로 흐르도록 함

            total_loss += loss.item()

            elapsed = time.time() - start_time
            # print('| epoch {:3d} | {:5d}/{:5d} batches | '
            # 'lr {:02.2f} |'
            # 'loss {:5.2f}'.format(t, b, len(train_x) / batch_size, scheduler_x.get_lr()[0], total_loss))

#        print('step : ' , t , 'loss : ' , loss.item())

def evaluate(eval_x_model, eval_y_model):
    eval_x_model.eval() # Turn on the evaluation mode
    eval_y_model.eval()

    total_loss = 0.

    with torch.no_grad():
        for b in range(0,len(df_train_x),batch_size):
            inpt = df_train_x[b:b+batch_size,:,:]
            target = df_train_y[b:b+batch_size]

            # x, y 트레인셋
            x_batch = torch.tensor(inpt[:,:,0],dtype=torch.float32, device = device) # 텐서로 바꾸면서
            y_batch = torch.tensor(inpt[:,:,1],dtype=torch.float32, device = device)
            
            if x_batch.shape == (batch_size,n_timesteps):
                x_batch = x_batch.reshape(batch_size,n_timesteps,1) # 3 채널로 맞춰줘야 함
                y_batch = y_batch.reshape(batch_size,n_timesteps,1)

                eval_x_model.init_hidden(x_batch.size(0))
                eval_y_model.init_hidden(y_batch.size(0))

                # x,y의 예측 값
                output_x = eval_x_model(x_batch)
                output_y = eval_y_model(y_batch)

                # x,y의 정답
                target_x_batch = torch.tensor(target[:, 0],dtype=torch.float32, device = device)
                target_y_batch = torch.tensor(target[:, 1],dtype=torch.float32, device = device)


                total_loss += criterion(output_x.view(-1), target_x_batch).item() + criterion(output_y.view(-1), target_y_batch).item()

    return total_loss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
cd DeepLearning_14Team

/content/drive/My Drive/DeepLearning_14Team


In [ ]:
import time

best_val_loss = float("inf")
epochs = 10 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(mv_net_x, mv_net_y)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f}'.format(epoch, (time.time() - epoch_start_time), val_loss))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_x = mv_net_x
        best_model_y = mv_net_y

        PATH_x = "lstm_checkpoint/LSTM + MSE x episode : {}, batch_size : {}, layer : {}, hidden : {}, epoch : {}".format(epoch,batch_size,layer,hidden, epoch) + ".pt"
        torch.save({
        'epoch': epoch,
        'model_state_dict': mv_net_x.state_dict(),
        'optimizer_state_dict': optimizer_x.state_dict(),
        'loss': best_val_loss,
        }, PATH_x)

        PATH_y = "lstm_checkpoint/LSTM + MSE y episode : {}, batch_size : {}, layer : {}, hidden : {}, epoch : {}".format(epoch,batch_size,layer,hidden, epoch) + ".pt"
        torch.save({
        'epoch': epoch,
        'model_state_dict': mv_net_y.state_dict(),
        'optimizer_state_dict': optimizer_x.state_dict(),
        'loss': best_val_loss,
        }, PATH_y)

    scheduler_x.step()
    scheduler_y.step()

In [ ]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data), scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.

    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [ ]:
val_x
test_x

In [ ]:
best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_x)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()